In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import json
import time
from datetime import datetime
import single_node_profiles as snp
import profiler
import end_to_end_profiles as e2e_profs
import numpy as np
from optimizer import BruteForceOptimizer
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
profs = snp.load_single_node_profiles()

In [4]:
profs.keys()

dict_keys(['alexnet', 'res152', 'res18', 'res50', 'inception', 'tf-kernel-svm', 'tf-lang-detect', 'tf-log-reg', 'tf-lstm', 'tf-nmt', 'tf-resnet-feats'])

In [5]:
dag = profiler.get_logical_pipeline("pipeline_three")
with open(os.path.abspath("../results/e2e_profs/systemx/resnet_cascade/slo_500ms/alex_1-r50_1-r152_2-171025_083730.json")) as f:
    sample_run = json.load(f)
scale_factors = profiler.get_node_scale_factors(sample_run, dag.reference_node)
node_configs = profiler.get_node_configs_from_experiment(sample_run)
node_profs = {name : profiler.NodeProfile(name, profs[name]) for name, _ in node_configs.items()}

In [15]:
opt = BruteForceOptimizer(dag, scale_factors, node_profs)

In [17]:
opt.select_optimal_config("gcp", 0.7, 10)

Processed 500
Processed 1000
Processed 1500
Processed 2000
Processed 2500
Processed 3000
Processed 3500
Processed 4000
Processed 4500
Processed 5000
Processed 5500
Processed 6000
Processed 6500
Processed 7000
Processed 7500
Processed 8000
Processed 8500
Processed 9000
Processed 9500
Processed 10000
Processed 10500
Processed 11000
Processed 11500
Processed 12000
Processed 12500
Processed 13000
Processed 13500
Processed 14000
Processed 14500
Processed 15000
Processed 15500
Processed 16000
Processed 16500
Processed 17000
Processed 17500
Processed 18000
Processed 18500
Processed 19000
Processed 19500
Processed 20000
Processed 20500
Processed 21000
Processed 21500
Initializing config to {'alexnet': NodeConfig(alexnet, 1, k80, 1.0, 1, gcp), 'res50': NodeConfig(res50, 1, k80, 1.0, 1, gcp), 'res152': NodeConfig(res152, 1, k80, 1.0, 1, gcp)} ({'latency': 0.6904060599999999, 'throughput': 39.84377879696414, 'cost': 2.1240000000000001})
Updating config to {'alexnet': NodeConfig(alexnet, 1, k80, 1

({'alexnet': NodeConfig(alexnet, 1, p100, 10.856742810603105, 1, gcp),
  'res152': NodeConfig(res152, 1, p100, 10.72030162514243, 1, gcp),
  'res50': NodeConfig(res50, 1, p100, 11.174747827156555, 1, gcp)},
 {'cost': 6.3659999999999997,
  'latency': 0.67181903,
  'throughput': 228.72770556062864})

In [ ]:
import itertools
def brute_force_optimizer(dag, scale_factors, node_profs, cost_constraint, latency_constraint):
    """
        This doesn't loo
    """
    all_node_configs = [node_profs[node].enumerate_configs(max_replication_factor=3) for node in dag.nodes()]     
    all_pipeline_configs = itertools.product(*all_node_configs)
    num_valid_configs = 0
    best_config = None
    best_config_perf = None
    cur_index = 0
    for p_config in all_pipeline_configs:
        cur_index += 1
        if cur_index % 500 == 0:
            print("Processed {}".format(cur_index))
        cur_node_configs = {n.name: n for n in p_config}
        if not profiler.is_valid_pipeline_config(cur_node_configs):
            continue
        cur_config_perf = profiler.estimate_pipeline_performance_for_config(
            dag, scale_factors, cur_node_configs, node_profs)
        if cur_config_perf["latency"] <= latency_constraint and cur_config_perf["cost"] <= cost_constraint:
            if best_config is None:
                best_config = cur_node_configs
                best_config_perf = cur_config_perf
                print("Initializing config to {} ({})".format(best_config, best_config_perf))
            else:
                if cur_config_perf["throughput"] > best_config_perf["throughput"]:
                    best_config = cur_node_configs
                    best_config_perf = cur_config_perf
                    print("Updating config to {} ({})".format(best_config, best_config_perf))
        
    return best_config, best_config_perf

In [ ]:
start = datetime.now()
brute_force_optimizer(dag, scale_factors, node_profs, 7.0, 0.8)
end = datetime.now()
print("{}".format((end-start).total_seconds()))

In [ ]:
profiler.estimate_pipeline_performance_for_config(dag, scale_factors, node_configs, node_profs)

In [ ]:
profs["alexnet"].columns